### 1. What is the purpose of this experiment?
LGBMをアンサンブルの1つとするにあたり、ベンチマークを作成する。
F1 Scoreと学習に使う時間を確認する。
また、TFIDFとEmbeddingの2通りを試す
### 2. Why do you this?
多様なモデルを作る為
### 3. Where are the points of technology and techniques?
LGBM, TFIDF or Word Embedding
### 4. How do you validate the effectiveness?
処理時間
F1 Score
### 5. What will you do next?
NNとアンサンブルしてLBを確認する。学習時間を確認する

In [8]:
is_local = False

%load_ext autoreload
%autoreload 2
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
from contextlib import contextmanager
import datetime
import sys
import re
import gc
import glob
import pickle as pkl
import os

from joblib import Parallel, delayed
from multiprocessing import Pool
import multiprocessing
def pararell_process(func, arg_list, cpu_cnt=multiprocessing.cpu_count()):
    process = Pool(cpu_cnt)
    callback = process.map_async(func, arg_list).get(600)
    process.close()
    process.terminate()
    return callback

def mkdir_func(path):
    try:
        os.stat(path)
    except:
        os.mkdir(path)

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
def logger_func():
    logger = getLogger(__name__)
    log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
    handler = StreamHandler()
    handler.setLevel('INFO')
    handler.setFormatter(log_fmt)
    logger.addHandler(handler)

    mkdir_func('../output')
    handler = FileHandler('../output/py_train.py.log', 'a')
    handler.setLevel(DEBUG)
    handler.setFormatter(log_fmt)
    logger.setLevel(DEBUG)
    logger.addHandler(handler)

    logger.info('start')

    return logger
logger = logger_func()


@contextmanager
def timer(name):
    """
    Taken from Konstantin Lopuhin https://www.kaggle.com/lopuhin
    in script named : Mercari Golf: 0.3875 CV in 75 LOC, 1900 s
    https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s
    """
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    
# NLP
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

key = 'qid'
qt = 'question_text'
target = 'target'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


SystemExit: 

### Cleansing Method

In [9]:
# Contraction replacement patterns
cont_patterns = [
    (b'(W|w)on\'t', b'will not'),
    (b'(C|c)an\'t', b'can not'),
    (b'(I|i)\'m', b'i am'),
    (b'(A|a)in\'t', b'is not'),
    (b'(\w+)\'ll', b'\g<1> will'),
    (b'(\w+)n\'t', b'\g<1> not'),
    (b'(\w+)\'ve', b'\g<1> have'),
    (b'(\w+)\'s', b'\g<1> is'),
    (b'(\w+)\'re', b'\g<1> are'),
    (b'(\w+)\'d', b'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

def prepare_for_char_n_gram(text):
    """ Simple text clean up process"""
    # 1. Go to lower case (only good for english)
    # Go to bytes_strings as I had issues removing all \n in r""
    clean = bytes(text.lower(), encoding="utf-8")
    # 2. Drop \n and  \t
    clean = clean.replace(b"\n", b" ")
    clean = clean.replace(b"\t", b" ")
    clean = clean.replace(b"\b", b" ")
    clean = clean.replace(b"\r", b" ")
    # 3. Replace english contractions
    for (pattern, repl) in patterns:
        clean = re.sub(pattern, repl, clean)
    # 4. Drop puntuation
    # I could have used regex package with regex.sub(b"\p{P}", " ")
    exclude = re.compile(b'[%s]' % re.escape(bytes(string.punctuation, encoding='utf-8')))
    clean = b" ".join([exclude.sub(b'', token) for token in clean.split()])
    # 5. Drop numbers - as a scientist I don't think numbers are toxic ;-)
    clean = re.sub(b"\d+", b" ", clean)
    # 6. Remove extra spaces - At the end of previous operations we multiplied space accurences
    clean = re.sub(b'\s+', b' ', clean)
    # Remove ending space if any
    clean = re.sub(b'\s+$', b'', clean)
    # 7. Now replace words by words surrounded by # signs
    # e.g. my name is bond would become #my# #name# #is# #bond#
    # clean = re.sub(b"([a-z]+)", b"#\g<1>#", clean)
#     clean = re.sub(b" ", b"# #", clean)  # Replace space
#     clean = b"#" + clean + b"#"  # add leading and trailing #

    return str(clean, 'utf-8')

def count_regexp_occ(regexp="", text=None):
    """ Simple way to get the number of occurence of a regex"""
    return len(re.findall(regexp, text))

def cleansing_text(text, remove_stopwords=True):

    # Convert words to lower case and split them
    text = re.sub("_", " ", text, flags=re.IGNORECASE)
    text = text.lower().split()
    regex_num = re.compile(u"[0-9０-９]")

    # Optionally, remove stop words
    if remove_stopwords:
        stops = STOPWORDS
        text = [w for w in text if (not w in stops) and not(regex_num.match(w))]

    text = " ".join(text)

    # Clean the text
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub("\'ve", " have ", text)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("é", "e", text)
    text = re.sub("’", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    text = re.sub("…", " ", text)
    text = re.sub("？", "?", text)
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\W|^)([0-9]+)[kK](\W|$)", r"\1\g<2>000\3", text) # better regex provided by @armamut
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"what's", "what is ", text)

    # Original
    text = re.sub(r" u s ", " America ", text)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" qoura ", " Quora ", text)
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r"(?=[a-zA-Z])ig ", "ing ", text)
    
    return text

### Data Load

In [4]:
def quara_load_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    return train, test

train, test = quara_load_data()
# Load id Text List
train_id_list = list(train[key].values)
test_id_list = list(test[key].values)
train_text_list = list(train[qt].values)
test_text_list = list(test[qt].values)
id_list = train_id_list + test_id_list
text_list = train_text_list + test_text_list

raw_trn_idx = list(train.index)
raw_test_idx = list(test.index)
y = train[target]

del train, test
gc.collect()

print(train_text_list[:10])

['How did Quebec nationalists see their province as a nation in the 1960s?', 'Do you have an adopted dog, how would you encourage people to adopt and not shop?', 'Why does velocity affect time? Does velocity affect space geometry?', 'How did Otto von Guericke used the Magdeburg hemispheres?', 'Can I convert montra helicon D to a mountain bike by just changing the tyres?', 'Is Gaza slowly becoming Auschwitz, Dachau or Treblinka for Palestinians?', 'Why does Quora automatically ban conservative opinions when reported, but does not do the same for liberal views?', 'Is it crazy if I wash or wipe my groceries off? Germs are everywhere.', 'Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?', 'Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disregarding their feelings/lives so you get to have something go your way and feel temporarily at ease. How did things change?']


### Cleansing Start

In [17]:
# Cleansing
with timer("Cleansing Dataset"):
    # 並列処理でクレンジング
    train_text_list = Parallel(n_jobs=-1)( [delayed(cleansing_text)(args) for args in train_text_list] )
    train_text_list = Parallel(n_jobs=-1)( [delayed(prepare_for_char_n_gram)(args) for args in train_text_list] )
    test_text_list = Parallel(n_jobs=-1)( [delayed(cleansing_text)(args) for args in test_text_list] )
    test_text_list = Parallel(n_jobs=-1)( [delayed(prepare_for_char_n_gram)(args) for args in test_text_list] )
    text_list = train_text_list + test_text_list

[Cleansing Dataset] done in 56 s


### TFIDF
ngramはいくつかパターンを試す
実験のため、tfidf matrixとvecotorizerは保存しておく

In [18]:
ngram_range=(1,1)
# ngram_range=(1,2)

def get_tfidf(text_list):
    '''
    Explain:
        テキストを1要素としてもったリストを渡し、TFIDFベクトルをもったsparse matrixを出力する
    Args:
        text_list(list): split前のテキストリスト. 1テキストが1つのTFIDFベクトルに変換される
    Return:
        sparse csr_matrix: TFIDF値が入ったスパースな行列
    '''
    # Get the tfidf
    with timer("Fitting TFIDF"):
        # TFIDFに変換するオブジェクトを作成する
        tfidf_vectorizer = TfidfVectorizer(
            max_features = 10000,
            min_df=10,
            max_df=0.5,
            stop_words="english",
            analyzer='word',
            #  analyzer='char',
            strip_accents='unicode',
            ngram_range=ngram_range,
            use_idf=True,
            smooth_idf=True,
            sublinear_tf=True
        ).fit(text_list)

    # Train, Testのテキストが順番に並んだリストを渡すと、各テキストをTFIDFベクトルに変換して返してくれる
    # テキストに対応するインデックスを渡して返ってくるのが対応するTFIDFベクトル
    with timer("Transform TFIDF"):
        csr_tfidf = tfidf_vectorizer.transform(text_list)
    return csr_tfidf, tfidf_vectorizer

# TFIDF
csr_tfidf, tfidf_vectorizer = get_tfidf(text_list)

# Kernel実行の場合は分岐
if is_local:
    HOME = os.path.expanduser("~")
    sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
    import utils
    utils.to_pkl_gzip(obj=tfidf_vectorizer, path='../input/bench_vectorizer_tfidf30000.gz')
    utils.to_pkl_gzip(obj=csr_tfidf, path='../input/bench_csr_tfidf30000.gz')
    sys.path.append(f'{HOME}/kaggle/data_analysis/model')
    from params_lgbm import params_quara
else:
    del text_list
    gc.collect()

[Fitting TFIDF] done in 11 s
[Transform TFIDF] done in 10 s


### LGBM

In [32]:
st_time = time.time()
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

def f1_calculation(y_val, y_pred):
    for thresh in np.arange(0.1, 0.401, 0.01):
        thresh = np.round(thresh, 2)
        f1 = f1_score(y_val, (y_pred>thresh).astype(int))
        logger.info(f"F1 score at threshold {thresh} is {f1}")
        

# Variables
tfidf_vocablary = tfidf_vectorizer.vocabulary_.items()
feature_list = np.array(tfidf_vectorizer.get_feature_names())
del tfidf_vectorizer
gc.collect()

# LGBM Args
model_type = 'lgb'
fold_type = 'stratified'
metric = 'accuracy'
fold = 2
learning_rate = 0.1
early_stopping_rounds = 100
num_boost_round = 10000
seed = 1208
params = {
    'num_threads': -1,
    'metric': 'binary_logloss',
    'objective': 'binary',
    'boosting_type':'gbdt',
    'bagging_freq': 1,
    'sigmoid': 1.1,
    'subsample': 0.9,
    'colsample_bytree': 0.2,
    'lambda_l1': 1,
    'lambda_l2': 5,
    'learning_rate': 0.1,
    'num_leaves': 100,
    'max_depth': 9,
    'bagging_seed': 1208,
    'data_random_seed': 1208,
    'feature_fraction_seed': 1208,
    'random_seed': 1208,
    'verbose': 1
}

# Result
select_features = [] # Feature Importanceによって選択したfeature群を入れるリスト

with timer("LGBM Setting"):
    
    # testも結合されてるので、trainのみにする
    tmp_train = csr_tfidf[raw_trn_idx]

    ' KFold '
    if fold_type == 'stratified':
        folds = StratifiedKFold(n_splits=fold, shuffle=True, random_state=seed)
        kfold = folds.split(tmp_train, y)
    
cv_feim = pd.DataFrame() # Feature Importanceの結果ファイルを入れるDF
for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    
    with timer(f"Validation: {n_fold} | LGBM Train"):
        x_train, y_train = tmp_train[trn_idx], y[trn_idx]
        x_val, y_val = tmp_train[val_idx], y[val_idx]
    
        if n_fold==0:
            prediction = np.zeros(len(x_val))
        
        # Dataset
        lgb_train = lgb.Dataset(data=x_train, label=y_train)
        lgb_eval = lgb.Dataset(data=x_val, label=y_val)
        
        estimator = lgb.train(
            train_set=lgb_train,
            valid_sets=lgb_eval,
            params=params,
            verbose_eval=200,
            early_stopping_rounds=early_stopping_rounds,
            num_boost_round=num_boost_round
        )
    
    with timer(f"Validation: {n_fold} | Prediction & Get F1 score"):
        y_pred = estimator.predict(x_val)
        score = log_loss(y_val, y_pred)
        logger.info(f'Fold No: {n_fold} | {metric}: {score}')
        logger.info(f"Train Shape: {x_train.shape}")
        
        prediction += y_pred
        
        f1_calculation(y_val, y_pred)
            
        ' Feature Importance '
        if len(cv_feim):
            cv_feim[f'{n_fold}_importance'] = estimator.feature_importance(importance_type='gain')
        else:
            feim_name = f'{n_fold}_importance'
            feim = pd.Series(estimator.feature_importance(importance_type='gain'), name=feim_name, index=feature_list).to_frame().reset_index().rename(columns={'index':'feature'})
            cv_feim = feim.copy()
            
prediction /= n_fold+1
f1_calculation(y_val, prediction)
    
with timer("Save Feature Importance"):
    col_feim = [col for col in cv_feim.columns if col.count('importance')]
    cv_feim['avg_importance'] = cv_feim[col_feim].mean(axis=1)
    cv_feim.sort_values(by='avg_importance', ascending=False, inplace=True)
    
    from matplotlib import pyplot as plt
    import japanize_matplotlib
    %matplotlib inline
    import seaborn as sns
    plt.figure(figsize=(8, 12))
    display(cv_feim.head())
    sns.barplot(data=cv_feim.iloc[:50, ], x='avg_importance', y='feature')
    plt.show()
    if is_local:
        cv_feim.to_csv(f'../valid/{start_time[4:12]}_{model_type}_TFIDF_f1{f1}_logloss{score}_lr{learning_rate}.csv', index=False)
    tmp_features = list(cv_feim[cv_feim['avg_importance']>30]['feature'].values)
    print(f'Selece Feature: {len(tmp_features)}')
    select_features += tmp_features
        
select_features = list(set(select_features))
print(len(select_features))
print(sorted(select_features))
print(f'All done in {time.time() - st_time:.0f} s')

MemoryError: 

### Feature Importance 上位のFeature Set

In [38]:
id_list = []
[id_list.append(id)  for word, id in tfidf_vocablary if word in select_features]
print(len(id_list))
print(len(select_features))
# sparse matrixのcolumn No=id No
select_mx = csr_tfidf.T[id_list].T
print(select_mx.shape)

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>